In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

from datetime import datetime, timedelta
import math
import time

import warnings
warnings.filterwarnings("ignore")

In [8]:
start_date = datetime.now() - timedelta(1)
year = str(start_date.year)
month = str(start_date.month)
day = "0" + str(start_date.day) if start_date.day < 10 else str(start_date.day)

In [20]:
required_entries = 1000

In [21]:
option = webdriver.ChromeOptions()
option.add_argument("start-maximized")

In [22]:
links_lst = []
i = 0
while i < required_entries:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
    time.sleep(5)
    driver.get(f"https://www.euronews.com/{year}/{month}/{day}")
    len_articles = driver.find_element(By.XPATH, '//p[@class="c-block-listing__results"]/strong[1]').text
    pages = int(len_articles) / 30
    pages = math.ceil(pages)
    for k in range(pages):
        driver.get(f"https://www.euronews.com/{year}/{month}/{day}?p={k+1}")
        articles_webscrapped = driver.find_elements(By.CLASS_NAME, "m-object__title__link ")
        for article in articles_webscrapped :
            links_lst.append(article.get_attribute('href'))
    driver.quit()
    i += int(len_articles)
    print(f"Added {i} links to the list")
    start_date -= timedelta(1)
    year = str(start_date.year)
    month = str(start_date.month)
    day = "0" + str(start_date.day) if start_date.day < 10 else str(start_date.day)
print(links_lst)

Added 30 links to the list
Added 58 links to the list
Added 119 links to the list
Added 177 links to the list
Added 245 links to the list
Added 305 links to the list
Added 354 links to the list
Added 372 links to the list
Added 396 links to the list
Added 449 links to the list
Added 514 links to the list
Added 582 links to the list
Added 651 links to the list
Added 702 links to the list
Added 721 links to the list
Added 749 links to the list
Added 819 links to the list
Added 877 links to the list
Added 936 links to the list
Added 993 links to the list
Added 1042 links to the list
['https://www.euronews.com/business/2024/06/11/single-market-shake-up-a-fifth-freedom-for-the-eus-internal-trading-arrangements', 'https://www.euronews.com/2024/06/11/natos-eastern-flank-countries-gather-in-riga-ahead-of-crucial-july-summit', 'https://www.euronews.com/green/2024/06/11/super-election-year-what-are-candidates-in-the-uk-us-and-australia-planning-on-the-climate', 'https://www.euronews.com/culture/

In [23]:
from bs4 import BeautifulSoup

In [24]:
errors = []
list_of_articles = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
time.sleep(5)
for article in links_lst:
    try :  
        driver.get(article)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        title =  driver.find_elements(By.CLASS_NAME, "c-article-redesign-title")[0].text
        if not title:
            title =  driver.find_elements(By.CLASS_NAME, "c-article-redesign-title")[1].text

        contributors_div = soup.find('div', class_='c-article-contributors')
        bold_tags = contributors_div.find_all('b')
        authors = [tag.text.strip() for tag in bold_tags]
        
        if not authors:
            contributors_div = soup.find('div', class_='c-article-contributors')
            author_link = contributors_div.find('a')
            authors = author_link.text.strip()
               
        date = driver.find_elements(By.CLASS_NAME, "c-article-publication-date")[0].get_attribute('datetime') 
        
        divs = soup.find_all('div', class_='u-flex-container u-flex--wrap u-padding-bottom-2')
        try:
            second_div = divs[1]
        except:
            second_div = divs[0]
            
        a_tags = second_div.find_all('a')
        tags = [tag.get_text(strip=True) for tag in a_tags]

        dictionnary_of_article = {"title": title, "authors": authors, "date": date, "tags": tags, "link": article }
        list_of_articles.append(dictionnary_of_article)
        time.sleep(4)

    except Exception as e: 
        errors.append({article: e})

print(f"articles:\n{list_of_articles}\nerrors:{errors}")
driver.quit()

list_of_articles:
[{'title': "'I start from a big, red alarm' - Enrico Letta urges radical EU single market shake-up", 'authors': 'Andrea Bolitho', 'date': '2024-06-11', 'tags': ['cryptocurrency', 'Football', 'Barcelona Football Club'], 'link': 'https://www.euronews.com/business/2024/06/11/single-market-shake-up-a-fifth-freedom-for-the-eus-internal-trading-arrangements'}, {'title': "NATO's eastern flank countries gather in Riga ahead of crucial July summit", 'authors': "Alessio Dell'Anna", 'date': '2024-06-11', 'tags': ['Marine Le Pen', 'Legislative election', 'France', 'Les Républicains', 'Emmanuel Macron'], 'link': 'https://www.euronews.com/2024/06/11/natos-eastern-flank-countries-gather-in-riga-ahead-of-crucial-july-summit'}, {'title': 'Super election year: What are candidates in the UK, US and Australia planning on the climate?', 'authors': 'Rosie Frost', 'date': '2024-06-11', 'tags': ['packaging', 'czechia', 'European Commission', 'EU Policy', 'competition'], 'link': 'https://www.

In [25]:
import pandas as pd

df = pd.DataFrame(list_of_articles, columns=["title", "authors", "date", "tags", "link"])

df.to_csv('articles.csv', index=False)

print("Dataset written")

Dataset written


In [26]:
df.head(10)

,title,authors,date,tags,link
0,"'I start from a big, red alarm' - Enrico Letta...",Andrea Bolitho,2024-06-11,"[cryptocurrency, Football, Barcelona Football ...",https://www.euronews.com/business/2024/06/11/s...
1,NATO's eastern flank countries gather in Riga ...,Alessio Dell'Anna,2024-06-11,"[Marine Le Pen, Legislative election, France, ...",https://www.euronews.com/2024/06/11/natos-east...
2,Super election year: What are candidates in th...,Rosie Frost,2024-06-11,"[packaging, czechia, European Commission, EU P...",https://www.euronews.com/green/2024/06/11/supe...
3,King Charles III's portrait targeted by animal...,David Mouriquand,2024-06-11,"[United Kingdom, Glastonbury, Gender equality,...",https://www.euronews.com/culture/2024/06/11/ki...
4,"'I start from a big, red alarm' - Enrico Letta...",Andrea Bolitho,2024-06-11,"[cryptocurrency, Football, Barcelona Football ...",https://www.euronews.com/business/2024/06/11/s...
5,NATO's eastern flank countries gather in Riga ...,Alessio Dell'Anna,2024-06-11,"[Marine Le Pen, Legislative election, France, ...",https://www.euronews.com/2024/06/11/natos-east...
6,Super election year: What are candidates in th...,Rosie Frost,2024-06-11,"[packaging, czechia, European Commission, EU P...",https://www.euronews.com/green/2024/06/11/supe...
7,King Charles III's portrait targeted by animal...,David Mouriquand,2024-06-11,"[United Kingdom, Glastonbury, Gender equality,...",https://www.euronews.com/culture/2024/06/11/ki...
8,Right-wing New Flemish Alliance wins Belgian '...,Clara Preve,2024-06-10,"[Flanders, Belgium, Far-right]",https://www.euronews.com/my-europe/2024/06/09/...
9,Poland exit polls: PM Tusk's coalition maintai...,Cynthia Kroet,2024-06-09,"[European Parliament, Poland, Jaroslaw Kaczyns...",https://www.euronews.com/my-europe/2024/06/09/...


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    1252 non-null   object
 1   authors  1252 non-null   object
 2   date     1252 non-null   object
 3   tags     1252 non-null   object
 4   link     1252 non-null   object
dtypes: object(5)
memory usage: 49.0+ KB
